# Importing the libraries

In [ ]:
import numpy as np
import random
import os
import cv2
import matplotlib.pyplot as plt

# Setting up the data directories

In [ ]:
data_dir = "data"

In [ ]:
CATEGORIES = ["with_mask", "without_mask"]

# Loading the data

In [ ]:
data = []
for category in CATEGORIES:
    path = os.path.join(data_dir, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, (75, 75))
        data.append([new_img_array, class_num])

# Shuffling the data

In [ ]:
random.shuffle(data)

# Length of Data

In [ ]:
print(f"Length of data : {len(data)}")

# Splitting the data into features and labels

In [ ]:
x_train = []
y_train = []

for feature, label in data:
    x_train.append(feature)
    y_train.append(label)

# Converting the data into numpy arrays

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

# Reshaping the data

In [ ]:
x_train = x_train.reshape(-1, 75, 75, 1)

# Normalizing the data

In [ ]:
x_train = x_train / 255.0

# Splitting the data into test and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

# Building the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

In [ ]:
model = Sequential()

In [ ]:
model.add(Input(shape=(75, 75, 1)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

# Compiling the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary

In [ ]:
model.summary()

# Training the model

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=32)

# Evaluating the model

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

# Testing the model

In [ ]:
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Plotting test results

In [ ]:
img_num = 34
print(f"Predicted label : {CATEGORIES[y_test[img_num]]}")
plt.imshow(x_test[img_num], cmap="gray")
plt.show()

# Saving the model

In [ ]:
model.save('face_mask_detector.h5')